In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import random
import numpy as np
import pandas as pd

In [2]:
import data.data_retriever as data_retriever

In [3]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(0)

In [4]:
df = data_retriever.get_data('USDT')
df['unix_time'] = pd.to_datetime(df['date-time']).astype(int) / 10**9

cols_to_keep = [
    'unix_time',
    'liquidityRate_avg',
    'variableBorrowRate_avg',
    'utilizationRate_avg',
    'close_price',
    'volume',
]

crypto_df = df[cols_to_keep]
crypto_df.head()

,unix_time,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,close_price,volume
0,1.678752e+09,0.026841,0.033669,0.885642,1.00450,5.479494e+06
1,1.678756e+09,0.018498,0.023178,0.888167,1.00421,7.850737e+06
2,1.678759e+09,0.011905,0.014973,0.883226,1.00463,7.511133e+06
3,1.678763e+09,0.060175,0.076011,0.878935,1.00520,7.736502e+06
4,1.678766e+09,0.026422,0.034169,0.859224,1.00480,8.454853e+06


# Dataloader

In [5]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data: pd.DataFrame, window_size: int, target_columns, forecast_size: int = 1):
        self.window_size = window_size
        self.forecast_size = forecast_size
        self.target_columns = target_columns  
        
        data_array = data.values
    
        target_indices = [data.columns.get_loc(col) for col in target_columns]  

        self.X, self.y = [], []
        for i in range(len(data_array) - window_size - forecast_size + 1):
            self.X.append(data_array[i : i + window_size])
            
            if len(target_indices) == 1:
                self.y.append(data_array[i + window_size : i + window_size + forecast_size, target_indices[0]].reshape(-1, 1))
            else:
                self.y.append(data_array[i + window_size : i + window_size + forecast_size, target_indices])

        self.X = np.array(self.X)
        self.y = np.array(self.y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (torch.tensor(self.X[idx], dtype=torch.float32), 
                torch.tensor(self.y[idx], dtype=torch.float32))

In [6]:
def create_dataloaders(
        data, 
        target_columns,
        window_size=168,
        forecast_size=1,
        batch_size=32, 
        val_size=0.1,
        test_size=0.1):
    total_size = len(data)
    train_size = int((1 - val_size - test_size) * total_size)
    val_size = int(val_size * total_size)
    
    train_data = data[:train_size]
    val_data = data[train_size:train_size + val_size]
    test_data = data[train_size + val_size:]
    
    train_dataset = TimeSeriesDataset(train_data, window_size, target_columns, forecast_size)
    val_dataset = TimeSeriesDataset(val_data, window_size, target_columns, forecast_size)
    test_dataset = TimeSeriesDataset(test_data, window_size, target_columns, forecast_size)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

# Model

In [7]:
class LSTMForecaster(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=2, forecast_size=1, dropout=0.2):
        super(LSTMForecaster, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.forecast_size = forecast_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :] 
        
        forecast = []
        for _ in range(self.forecast_size):
            out = self.fc(out)
            forecast.append(out.unsqueeze(1))
        
        forecast = torch.cat(forecast, dim=1) 
        return forecast

# Trainer

In [8]:
def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.001, device="cuda", patience=10, lr_patience=5, lr_decay_factor=0.1):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=lr_decay_factor, patience=lr_patience, verbose=True)

    best_val_loss = float("inf")
    epochs_without_improvement_es = 0
    epochs_without_improvement_lr = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)

                y_pred = model(X_val)
                loss = criterion(y_pred, y_val)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.8f}, Val Loss: {val_loss:.8f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement_es = 0
            epochs_without_improvement_lr = 0
        else:
            epochs_without_improvement_es += 1
            epochs_without_improvement_lr += 1

        if epochs_without_improvement_es >= patience:
            print(f"Early stopping at epoch {epoch+1} due to no improvement in validation loss for {patience} epochs.")
            break

        if epochs_without_improvement_lr >= lr_patience:
            print(f"Validation loss plateaued for {lr_patience} epochs. Reducing learning rate by factor of {lr_decay_factor}.")
            scheduler.step(val_loss)
            epochs_without_improvement_lr = 0

# Evaluator

In [9]:
def evaluate_model(model, test_loader, device="cuda"):
    model.to(device)
    model.eval()

    criterion = nn.MSELoss(reduction='none')
    total_samples = 0
    total_mape = torch.zeros(model.output_size, device=device) 
    total_mse = torch.zeros(model.output_size, device=device) 

    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)

            y_pred = model(X_test)

            mse_per_element = criterion(y_pred, y_test)
            mape_per_output = torch.mean(torch.abs((y_test - y_pred) / (y_test + 1e-8)), dim=(0, 1)) * 100

            total_mape += mape_per_output * X_test.size(0)
            total_samples += X_test.size(0)

            mse_per_output = mse_per_element.mean(dim=(0, 1)) 
            total_mse += mse_per_output * X_test.size(0)

    average_mse = total_mse.cpu().numpy() / total_samples
    average_mape = total_mape.cpu().numpy() / total_samples

    print(f"Test MSE (for each output dimension): {average_mse}")
    print(f"Test MAPE (for each output dimension): {average_mape}")

    return average_mse, average_mape


# Training

In [10]:
def train_and_evaluate(
        crypto_df,
        input_size=6, 
        hidden_size=32, 
        num_layers=4, 
        output_size=2, 
        forecast_size=1, 
        dropout=0.2,
        target_columns=['liquidityRate_avg', 'variableBorrowRate_avg'],
        window_size=168,
        batch_size=32,
        val_size=0.1,
        test_size=0.1,
        num_epochs=100,
        learning_rate=0.001,
        device="cuda" if torch.cuda.is_available() else "cpu",
        patience=10,
        lr_patience=5,
        lr_decay_factor=0.1,
):

    train_loader, val_loader, test_loader = create_dataloaders(
        crypto_df,
        target_columns=target_columns,
        window_size=window_size,
        forecast_size=forecast_size,
        batch_size=batch_size,
        val_size=val_size,
        test_size=test_size
    )
    
    model = LSTMForecaster(
        input_size, 
        hidden_size, 
        num_layers, 
        output_size, 
        forecast_size, 
        dropout
    ).to(device)
    
    train_model(
        model, 
        train_loader, 
        val_loader, 
        num_epochs=num_epochs, 
        learning_rate=learning_rate, 
        device=device, 
        patience=patience, 
        lr_patience=lr_patience, 
        lr_decay_factor=lr_decay_factor
    )

    evaluate_model(model, test_loader, device=device)

In [11]:
# window size of 1 day
train_and_evaluate(crypto_df, window_size=24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00191607, Val Loss: 0.00230264
Epoch 2/100, Train Loss: 0.00186930, Val Loss: 0.00235975
Epoch 3/100, Train Loss: 0.00186489, Val Loss: 0.00218279
Epoch 4/100, Train Loss: 0.00186761, Val Loss: 0.00220394
Epoch 5/100, Train Loss: 0.00185475, Val Loss: 0.00235606
Epoch 6/100, Train Loss: 0.00185826, Val Loss: 0.00222995
Epoch 7/100, Train Loss: 0.00185403, Val Loss: 0.00223126
Epoch 8/100, Train Loss: 0.00185839, Val Loss: 0.00225587
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 9/100, Train Loss: 0.00185263, Val Loss: 0.00223713
Epoch 10/100, Train Loss: 0.00185801, Val Loss: 0.00224372
Epoch 11/100, Train Loss: 0.00185463, Val Loss: 0.00238382
Epoch 12/100, Train Loss: 0.00185006, Val Loss: 0.00223432
Epoch 13/100, Train Loss: 0.00185138, Val Loss: 0.00230541
Early stopping at epoch 13 due to no improvement in validation loss for 10 epochs.
Test MSE (for each output dimension): [0.00044847 0.00088729]
Test MAPE (for e

In [12]:
# window size of 1 week
train_and_evaluate(crypto_df, window_size=7*24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00213524, Val Loss: 0.00228686
Epoch 2/100, Train Loss: 0.00188656, Val Loss: 0.00233357
Epoch 3/100, Train Loss: 0.00187718, Val Loss: 0.00257181
Epoch 4/100, Train Loss: 0.00188997, Val Loss: 0.00225524
Epoch 5/100, Train Loss: 0.00187924, Val Loss: 0.00249794
Epoch 6/100, Train Loss: 0.00186706, Val Loss: 0.00246717
Epoch 7/100, Train Loss: 0.00186754, Val Loss: 0.00239406
Epoch 8/100, Train Loss: 0.00186503, Val Loss: 0.00242032
Epoch 9/100, Train Loss: 0.00186556, Val Loss: 0.00261359
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 10/100, Train Loss: 0.00186986, Val Loss: 0.00241719
Epoch 11/100, Train Loss: 0.00186108, Val Loss: 0.00243264
Epoch 12/100, Train Loss: 0.00186482, Val Loss: 0.00247978
Epoch 13/100, Train Loss: 0.00186504, Val Loss: 0.00235851
Epoch 14/100, Train Loss: 0.00186639, Val Loss: 0.00253142
Early stopping at epoch 14 due to no improvement in validation loss for 10 epochs.
Test MSE (for each 

In [13]:
# window size of 1 month
train_and_evaluate(crypto_df, window_size=30*24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00216622, Val Loss: 0.00327930
Epoch 2/100, Train Loss: 0.00189674, Val Loss: 0.00373778
Epoch 3/100, Train Loss: 0.00189325, Val Loss: 0.00321490
Epoch 4/100, Train Loss: 0.00189153, Val Loss: 0.00389923
Epoch 5/100, Train Loss: 0.00189564, Val Loss: 0.00332547
Epoch 6/100, Train Loss: 0.00190431, Val Loss: 0.00318835
Epoch 7/100, Train Loss: 0.00188726, Val Loss: 0.00350815
Epoch 8/100, Train Loss: 0.00188963, Val Loss: 0.00362196
Epoch 9/100, Train Loss: 0.00188924, Val Loss: 0.00336556
Epoch 10/100, Train Loss: 0.00189732, Val Loss: 0.00362094
Epoch 11/100, Train Loss: 0.00188465, Val Loss: 0.00367605
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 12/100, Train Loss: 0.00189110, Val Loss: 0.00357516
Epoch 13/100, Train Loss: 0.00188780, Val Loss: 0.00362915
Epoch 14/100, Train Loss: 0.00188540, Val Loss: 0.00360895
Epoch 15/100, Train Loss: 0.00188739, Val Loss: 0.00340196
Epoch 16/100, Train Loss: 0.00188637, Val L